## Selectively stretch and decompress segments
<p>The goal is to enable selectively staying on one place for a longer time. This can be accomplished by selecting a segment of an array to double in size. </P>
<p>The assumption is that we will take the segmented chorale numpy arrays in directory segmented_chorales, and load them into this notebook and mess with them. This notebook will no longer need to synthesize arrays using the model, since that is done in the notebeeks called coconet_incremental_synthesis.<p>

In [ ]:
import numpy as np
import copy
import mido
import time
from midi2audio import FluidSynth
from IPython.display import Audio, display
import os
import muspy
import piano 
import selective_stretching_codes
import subprocess
from numpy.random import default_rng
rng = np.random.default_rng(seed=42)

CSD_FILE = 'goldberg_aria1.csd'
LOGNAME = 'goldberg5.log'
root = 5
mode = 'major'

In [ ]:
def quick_play(chorale, gain, outfile = 'test.wav', outmidi='test.mid',bpm=60): # save the chorale to midi and put up a display to play it. Fast, simple, and out of tune.
    pad8=np.zeros((chorale.shape[0],8),dtype=int)
    chorale = np.concatenate((chorale,pad8),axis=1) # midi likes and ending to avoid the edge case bugs.
    midi_output = selective_stretching_codes.piano_roll_to_midi(chorale, bpm=bpm) # convert to mido object at 60 time_steps per minute, one per second.
    music = muspy.from_mido(midi_output) # convert mido to muspy music
    muspy.write_midi(outmidi, music) # write the midi file to disk
    muspy.write_audio(outfile, music, 'wav', 'font.sf2', 44100, gain) # generate a wav file using fluidsynth
    audio = Audio(outfile) # get ready to show the audio widget
    display(audio) # display the audio widget

## See if you can load one of the segmented fantasy chorales from the store of numpy arrays

Here's what this cell does:

- Load the numpy array, a (7, 16, 32) structure created by the coconet-model. Each of the 7 are independent 32 1/16th note sub-chorales based on taking an actual Bach chorale, BWV 180 'Schmücke dich, o liebe Seele'. A reminder that these are 16 voice chorales made by starting with three voices of Schmücke and using the coconet model to build the fourth voice. Then taking the new voice and two other existing voices and generating another. I kept doing that until I have four four voice chorales, or a 16 voice chorale, but one where each of the four have only a very limited knowledge of the other four chorales. It's sometimes a complex mess, but it holds somewhat together by the fact that they start with a real Bach chorale, and then slowly morph it into something artificial.
- Restore normal MIDI numbers
- expand and concatenate the segments
- arpeggiate it
- make a midi from the arpeggiated chorale
- save it
- use Csound to generate a wav file.  

In [ ]:
file_stub = 'chorale_96' # this one has the lowest entropy score of the 16 voice artificial chorales.
numpy_file = os.path.join('segmented_chorales', file_stub + '.npy')
chorale = np.load(numpy_file)+30 # load a generated chorale
concat_chorale = selective_stretching_codes.expand_and_concatenate(chorale)
print(f'chorale.shape: {chorale.shape}, concat_chorale.shape: {concat_chorale.shape}')
# play_chorale = transpose_up_segment(concat_chorale) # the transposition into F has already happened as has the +30
mask = selective_stretching_codes.set_mask(randomized=True)
arp_chorale = selective_stretching_codes.arpeggiate_and_stretch(concat_chorale,mask,3) # chorale array, mask array, and skip value maximum, could be lower
midi_output = selective_stretching_codes.piano_roll_to_midi(arp_chorale) # convert to mido object
music = muspy.from_mido(midi_output) # convert mido to muspy music
muspy.write_midi(file_stub + '.mid',music)

velocity = 67
volume = 15
tpq = 1.2 # time per quarter note = 1.2 seconds, so each 1/16th note gets .3 seconds. 
upsample = 3
# selective_stretching_codes.piano_roll_to_csound(arp_chorale,velocity,volume,tpq,upsample)

## Evaluate the chorales in segmented_chorale using muspy metrics. Search for the H with high entropy.
<p>This will load the chorales one at a time and run the muspy metrics against the finished decoded chorale. I don't put much stock in these metrics, except for the class entropy. That one gives extra value for violating classical theory rules. I like those with character.</p><p> 
<p>This measure is derived from a paper by Wu & Yang: The Jazz Transformer on the front line: exploring the shortcomings of ai-composed music through quantitative measures.
    <a href="https://arxiv.org/pdf/2008.01307.pdf"> Shih-Lun Wu and Yi-Hsuan Yang</a>
    
<blockquote>
        5.1 Pitch Class Histogram Entropy
To gain insight into the usage of different pitches, we first
collect the notes appeared in a certain period (e.g., a bar)
and construct the 12-dimensional pitch class histogram
−→h ,
according to the notes’ pitch classes (i.e. C, C#, ..., A#, B),
normalized by the total note count in the period such that
P
i
hi = 1. Then, we calculate the entropy of
−→h :
H(
−→h ) = −
X
11
i=0
hi
log2
(hi). (2)
The entropy, in information theory, is a measure of “uncertainty” of a probability distribution [40], hence we adopt
it here as a metric to help assessing the music’s quality in
tonality. If a piece’s tonality is clear, several pitch classes
should dominate the pitch histogram (e.g., the tonic and
the dominant), resulting in a low-entropy
−→h ; on the contrary, if the tonality is unstable, the usage of pitch classes
is likely scattered, giving rise to an
−→h with high entropy</blockquote>

In [ ]:
# the next line will generate a report on the chorales. 
# You can select which voices to include, and you will get a different report.
# For example [(0,16]) would use all the voices
# metrics = selective_stretching_codes.print_chorale_metric_report(pick_voices=[(12,16)])

## Looking for interesting places to stop and linger a while.

The chorales are interesting, but begin sound too conventional. As a composer, I like to sometimes take and existing piece and make my own variations on it. Bach did it, as did Brahms, Liszt, and many other composers. One technique that I have used in the past is to find expecially interesting parts of a simple piece and draw them out more, while keeping the overall forward moving structure of the piece. I call it the 'find an interesting place stop and linger a while' method of variations. It's just one method to create a variation. In this case, I call some functions that identify sections of the chorale that are especially interesting, as a measured by the number of notes that are not in the root key, in this case F major. I identify those, then stretch them out from whatever their current time steps to something much longer. 

In [ ]:
# pull in an interesting chorale from the collection of numpy segments
filename = os.path.join('segmented_chorales','chorale_90.npy')
voice_low = 8
voice_high = 16
print(filename)
play_chorale = selective_stretching_codes.transpose_up_segment(selective_stretching_codes.expand_and_concatenate(np.load(filename)),30) 
play_chorale = play_chorale[voice_low:voice_high,:] # this is where you need to pick the voices
print(f'play_chorale.shape: {play_chorale.shape}, play_chorale.dtype: {play_chorale.dtype}')

In [ ]:
# now double up the voices to become twice as many voices.
play_chorale = np.concatenate((play_chorale,play_chorale),axis = 0)
print(f'play_chorale.shape: {play_chorale.shape}, play_chorale.dtype: {play_chorale.dtype}')

## Make the chorale longer: factor = 1 2x as long, factor = 2, 3x as long

In [ ]:
# Before you do the discover, stretch the chorale out by making it 4 times as long.
start_decompress = 0
end_decompress = play_chorale.shape[1]
factor = 1
print(f'play_chorale.shape: {play_chorale.shape}')
play_chorale = selective_stretching_codes.decompress_segment(play_chorale,start_decompress,end_decompress,factor=factor) 
print(f'play_chorale.shape: {play_chorale.shape}, play_chorale.dtype: {play_chorale.dtype}')

print(f'Decompression by a factor of {factor} results in change by a factor of: {play_chorale.shape[1]/end_decompress}')

In [ ]:
# Now we have to find the the in tune time_steps, and challenging time_steos: range_in_tune and the range_of_steps
scores = selective_stretching_codes.assign_scores_to_time_steps(play_chorale)
range_of_steps = selective_stretching_codes.find_challenging_time_steps(scores)
range_in_tune = selective_stretching_codes.find_in_tune_time_steps(play_chorale, range_of_steps)


In [ ]:
# give me a set of random numbers from 2 up to but not including 8
factors = rng.integers(low=2,high=8,size=range_of_steps.shape[0])
print(f'factors: {factors}')

## Now that those segments have been decompressed, I need to refer to them to choose a theme to repeat.

We know where they were in the original, we also know their locations in the expanded set. 

In [ ]:
decom_chorale, challenging_steps, in_tune_steps = selective_stretching_codes.expand_challenging_time_steps(play_chorale, range_of_steps, range_in_tune)

# s = 0
# for steps in (challenging_steps):
#     print(f'in_tune_steps[{s}]:     {in_tune_steps[s,2:4]}, size: {in_tune_steps[s,3] - in_tune_steps[s,2]}')
#     print(f'challenging_steps[{s}]: {challenging_steps[s,2:4]}, size: {challenging_steps[s,3] - challenging_steps[s,2]}')
#     s += 1
# print(f'in_tune_steps[{s}]:     {in_tune_steps[s,2:4]}, size: {in_tune_steps[s,3] - in_tune_steps[s,2]}')    

In [ ]:
skip_time_step_in_arp = 3 # Arpeggiate one time, then skip the next three, then arpegiate the next, and so on.
mask = rng.integers(low=0, high=2, size=(16,16), dtype=np.uint8)
arp_chorale = selective_stretching_codes.arpeggiate_and_stretch(decom_chorale,mask,skip_time_step_in_arp)
tpq = 1.2 # time per quarter note.
upsamp = 4 # the higher the more mellow the sound
velocity_base = 67 # this cents the middle velocity sample from the Bosendorfer set
volume = 12 # avoid distortion. Slowly increse this until the overall amps gets close to but not over 32k.

In [ ]:
if selective_stretching_codes.valid_midi(arp_chorale): 
    # selective_stretching_codes.piano_roll_to_csound(arp_chorale,velocity_base,volume,tpq,upsamp)
    quick_play(arp_chorale, 2, outfile = 'test', outmidi='test.mid',bpm=60)

## Convolve with the impulse response from Teatro Alcorcon in Madrid from Angelo Farina Collection
<p>The next few cells require a great deal of installation work to accomplish. You need to install the following:
    
- Csound - available in most Linux repos with the operating system's standard program installer
    - sudu dnf install csound-devel
- sox 
- ffmpeg
   

In [ ]:
version = 15
!csound goldberg_aria1c.csd

In [ ]:
!ls -lth /home/prent/Music/sflib/goldberg_aria1a-c.wav

In [ ]:
!sox /home/prent/Music/sflib/goldberg_aria1a-c.wav save1.wav reverse
!sox save1.wav save2.wav silence 1 0.01 0.01
!sox save2.wav save1.wav reverse
!sox save1.wav /home/prent/Music/sflib/goldberg_aria1-t15.wav silence 1 0.01 0.01
!rm save1.wav
!rm save2.wav
!ls -lth /home/prent/Music/sflib/goldberg_aria1-t14.wav

In [ ]:
!ffmpeg -y -i /home/prent/Music/sflib/goldberg_aria1-t15.wav\
    -b:a 320k /home/prent/Music/sflib/goldberg_aria1-t15.mp3
audio = Audio('/home/prent/Music/sflib/goldberg_aria1-t15.mp3')
display(audio)